In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
# --- Cargar el dataset FINAL con features contextuales ---
DATA_PATH = '../data/processed/contextual_featured_dataset.csv'
df = pd.read_csv(DATA_PATH)

print("Dataset final con features contextuales cargado. Dimensiones:", df.shape)

Dataset final con features contextuales cargado. Dimensiones: (6280, 27)


In [3]:
# --- Definir Features (X) y Target (y) ---
# Ahora incluimos nuestras nuevas features: 'league_strength_factor' y 'club_tier'
TARGET = 'market_value_in_eur'
cols_to_exclude = [
    'player_id', TARGET, 'goals', 'assists', 'minutes_played', 'games_played',
    'name', 'position', 'sub_position', 'country_of_citizenship', 
    'current_club_name', 'contract_expiration_date', 'valuation_date',
    'player_club_domestic_competition_id', 'competition_code', 'competition_name' # Columnas intermedias
]

features = [col for col in df.columns if col not in cols_to_exclude]
X = df[features].select_dtypes(include=np.number)
y_log = np.log1p(df[TARGET])

print(f"\nNúmero de features para el modelo final: {len(X.columns)}")
print("Features:", X.columns.tolist())



Número de features para el modelo final: 9
Features: ['age', 'height_in_cm', 'minutes_per_90', 'goals_per_90', 'assists_per_90', 'goals_plus_assists_per_90', 'contract_months_remaining', 'league_strength_factor', 'club_tier']


In [4]:
# --- Cargar los mejores hiperparámetros que encontramos ---
best_params_300 = {
    'learning_rate': 0.1963845368531851, 'num_leaves': 226, 'max_depth': 3,
    'min_child_samples': 10, 'feature_fraction': 0.5368246241854336,
    'bagging_fraction': 0.9625685216118971, 'bagging_freq': 4,
    'lambda_l1': 2.52157740846641, 'lambda_l2': 0.04049748399375717,
    'objective': 'regression_l1', 'metric': 'mae', 'n_estimators': 2000,
    'verbose': -1, 'n_jobs': -1, 'seed': 42
}

In [5]:
# --- Entrenar y Evaluar el Modelo FINAL ---
X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.2, random_state=42)

final_model = lgb.LGBMRegressor(**best_params_300)

print("\nEntrenando modelo final con features contextuales...")
final_model.fit(X_train, y_train_log,
                eval_set=[(X_test, y_test_log)],
                eval_metric='mae',
                callbacks=[lgb.early_stopping(100, verbose=False)])

print("Entrenamiento completado.")


Entrenando modelo final con features contextuales...
Entrenamiento completado.


In [6]:
# --- Evaluación Final ---
preds_log = final_model.predict(X_test)
y_test_eur = np.expm1(y_test_log)
preds_eur = np.expm1(preds_log)

mae_final = mean_absolute_error(y_test_eur, preds_eur)
mae_anterior_optimizado = 3224641

print("\n--- ¡RESULTADOS FINALES DEFINITIVOS! ---")
print(f"MAE Modelo Anterior (Optimizado): €{mae_anterior_optimizado:,.0f}")
print(f"MAE Modelo FINAL (con Contexto):  €{mae_final:,.0f}")

improvement = mae_anterior_optimizado - mae_final
improvement_percent = (improvement / mae_anterior_optimizado) * 100

print("\n--- Conclusión Final del Modelado ---")
if improvement > 0:
    print(f"¡MEJORA MASIVA! El modelo con contexto es €{improvement:,.0f} más preciso.")
    print(f"Esto representa una reducción adicional del error del {improvement_percent:.2f}%.")
else:
    print(f"No se ha conseguido una mejora. El contexto no ha ayudado como se esperaba.")


--- ¡RESULTADOS FINALES DEFINITIVOS! ---
MAE Modelo Anterior (Optimizado): €3,224,641
MAE Modelo FINAL (con Contexto):  €2,921,504

--- Conclusión Final del Modelado ---
¡MEJORA MASIVA! El modelo con contexto es €303,137 más preciso.
Esto representa una reducción adicional del error del 9.40%.


In [7]:
# =============================================================================
# Guardar el Modelo FINAL y sus Columnas
# =============================================================================
import joblib
import json

# Guardar el objeto del modelo
MODEL_PATH = '../models/lgbm_final_context_model.pkl'
joblib.dump(final_model, MODEL_PATH)
print(f"Modelo final guardado en: {MODEL_PATH}")

# Guardar la lista de columnas que el modelo espera, en el orden correcto
# Esto es CRUCIAL para la producción
COLUMNS_PATH = '../models/model_columns.json'
with open(COLUMNS_PATH, 'w') as f:
    json.dump(X_train.columns.tolist(), f)
print(f"Columnas del modelo guardadas en: {COLUMNS_PATH}")

Modelo final guardado en: ../models/lgbm_final_context_model.pkl
Columnas del modelo guardadas en: ../models/model_columns.json
